# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [1]:
import Crypto.Util.number as cu
import hashlib as hl
import math
import random

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [2]:
nlen = 2048
print((nlen-1)//2)
print(nlen//2)
min_e = 65537
max_e = pow(2, 256)

1023
1024


In [3]:
def generate_numbers():
    def generate_e():
        return 65537
        # return random.randint(min_e, max_e)
    def check_diff(p, q):
        return abs(p - q) > pow(2, nlen//2 - 100)
    p = cu.getPrime(nlen//2)
    q = cu.getPrime(nlen//2)
    while not check_diff(p, q):
        p = cu.getPrime(nlen//2)
        q = cu.getPrime(nlen//2)
    f = (p - 1) * (q - 1)
    e = generate_e()
    while not math.gcd(f, e) == 1:
        e = generate_e()
    return p, q, f, e

p, q, f, e = generate_numbers()
while math.lcm(p - 1, q - 1) < e * pow(2, nlen // 2):
    p, q, f, e = generate_numbers()

n = p * q
d = cu.inverse(e, f)

private_key = (d, n)
public_key = (e, n)

print(f"{private_key=}")
print(f"{public_key=}")

private_key=(6789397741769217172522081841230759934926720945531779072556611369312534184017431887081231881270071466933436090851565746583330705631847871815429913630257749125767381875250788167800146537057635060044950728260197760741930861152895886663723181265052182670425469097159660075875023582736095874304370357486354006067697699269749777280308990598962480756672225287751862550966308914968907300723803614615197291084576033965260895607215072834140628161132807051699296651477811489976494048249964115193751295851979049474783104139260727165251237378282873965705546819249657902306490193540714720674384726511852492209575089415755600109433, 158201222997343804961807465558110045458043273343993530924462290873439363157914539423893441585293562443439024776412950413792129863825077855069270514679016605438177098043913426777045510772646742846535567047567581826688445867658869986589072790502639869043473642971113077719057605255555541248057142899304267402284653764257449456590525308329784366438595656343113990550360526

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


Czym to M jest i jakie ma być?

**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [4]:
def text_to_int(text):
    num = 0
    for letter in text:
        n = ord(letter)
        num *= 256
        num += n
    return num

def int_to_text(num):
    res = ""
    while num > 0:
        q = num % 256
        res += chr(q)
        num //= 256
    return res[::-1]


In [5]:
def encrypt(M, public_key):
    num = text_to_int(M)
    C = pow(num, public_key[0], public_key[1])
    return C

def decrypt(C, private_key):
    num = pow(C, private_key[0], private_key[1])
    M = int_to_text(num)
    return M

In [6]:
M = "Ala ma kota"

C = encrypt(M, public_key)
encrypted = int_to_text(C)
decrypted = decrypt(C, private_key)

print(encrypted)
print()
print(decrypted)

ç¸räåLðJBE²À§¡äB¸)µ»VµgD'%)ü ÛB2çCÎXl­ôJ°¶Êñà»Qóª1'§÷Ôevk@xårcÔ1:ÚdAØ7+Ì'»7tvNEsÖEU9céÆ{Éf`y³Æ
Ñò`ëOpªbN$v#'ªbA.i}p5¡Ä¦æâ-/Æ¾ºó}N»çJâ0oÒ.ÂòËû

Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [7]:
def generate_SIG(M):
    h = hl.sha256(M.encode("utf-8")).hexdigest()
    SIG = encrypt(h, private_key)
    return (M, SIG)

def verify_SIG(SIG):
    h = hl.sha256(SIG[0].encode("utf-8")).hexdigest()
    VER = decrypt(SIG[1], public_key)
    return VER == h

In [8]:
sig = generate_SIG(M)
verify_SIG(sig)

True

In [9]:
sig = generate_SIG(M)
sig = ("Ala nie ma kota", sig[1])
verify_SIG(sig)

False

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [10]:
H = hl.sha256
hlen = 256 // 8

def mgf1(M, len):
    if isinstance(M, str):
        M = bytes(M, "utf-8")
    output = b""
    for i in range(math.ceil(len / hlen)):
        tmp = H(M + i.to_bytes(4)).digest()
        output = output + tmp
    return output[:len]

In [11]:
mgf1("", 20)

b'\xdf?a\x98\x04\xa9/\xdb@W\x19-\xc4=\xd7H\xeaw\x8a\xdc'

In [12]:
mgf1("", 5)

b'\xdf?a\x98\x04'

In [13]:
mgf1(b"Lorem ipsum dolor sit amet, consectetur adipiscing elit.", 20)

b'aR\xccR\xf0\x89\xbc7\x19I\xf4W\xc7\x0f?km58\xdf'

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [14]:
def xor(n, m):
    xored = int.from_bytes(n) ^ int.from_bytes(m)
    s = math.ceil(xored.bit_length() / 8)
    return xored.to_bytes(s)

In [15]:
M = "Ala ma kota"

def oaep_encoding(M, k):
    if isinstance(M, str):
        M = bytes(M, "utf-8")
    mlen = len(M)
    if mlen > k - 2 * hlen - 2:
        raise ValueError("Message is too long.")
    lHash = H(b"").digest()
    PS = b'\x00' * (k - mlen - 2 * hlen - 2)
    DB = lHash + PS + b'\x01' + M
    assert len(DB) == k - hlen - 1
    seed = b''.join([random.randint(0, 255).to_bytes(1) for _ in range(hlen)])
    dbMask = mgf1(seed, k - hlen - 1)
    maskedDB = xor(DB, dbMask)
    seedMask = mgf1(maskedDB, hlen)
    maskedSeed = int.from_bytes(seed) ^ int.from_bytes(seedMask)
    s = math.ceil(maskedSeed.bit_length() / 8)
    maskedSeed = maskedSeed.to_bytes(s)
    EM = b'\x00' + maskedSeed + maskedDB
    assert len(EM) == k
    return EM

In [16]:
def oaep_decoding(EM, k, mlen=None):
    Y = EM[0:1]
    maskedSeed = EM[1:1+hlen]
    maskedDB = EM[1+hlen:]
    lHash = H(b"").digest()
    seedMask = mgf1(maskedDB, hlen)
    seed = int.from_bytes(maskedSeed) ^ int.from_bytes(seedMask)
    s = math.ceil(seed.bit_length() / 8)
    seed = seed.to_bytes(s)
    dbMask = mgf1(seed, k - hlen - 1)
    DB = xor(maskedDB, dbMask)
    lHashPrim = DB[:hlen]
    assert lHash == lHashPrim
    assert Y == b'\x00'
    # PS = DB[hlen:K - mlen - hlen - 2]
    # assert DB[K - mlen - hlen - 2:K - mlen - hlen - 1] == b'\x01'
    # M = DB[K - mlen - hlen - 1:]
    # return M
    M = DB[hlen:]
    while M[0:1] == b'\x00': # remove PS from M
        M = M[1:]
    assert M[0:1] == b'\x01'
    return M[1:]

In [17]:
# mlen = len(bytes(M, "utf-8"))

# em = oaep_encoding(M, nlen)

# message = oaep_decoding(em, nlen, mlen)
# print(message.decode("utf-8"))

In [18]:
em = oaep_encoding(M, nlen)

message = oaep_decoding(em, nlen)
print(message.decode("utf-8"))

Ala ma kota


In [19]:
def oaep_encrypt(M, public_key):
    k = math.ceil(public_key[1].bit_length() / 8)
    assert len(M) <= k - hlen  - 2
    oaep_M = oaep_encoding(M, k)
    num = int.from_bytes(oaep_M)
    C = pow(num, public_key[0], public_key[1])
    return C

def oaep_decrypt(C, private_key):
    k = math.ceil(private_key[1].bit_length() / 8)
    assert k >= 2 * hlen + 2
    num = pow(C, private_key[0], private_key[1])
    oaep_EM = num.to_bytes(k)
    M = oaep_decoding(oaep_EM, k)
    return M

In [20]:
M = "Ala ma kota"

C = oaep_encrypt(M, public_key)
encrypted = int_to_text(C)
decrypted = oaep_decrypt(C, private_key).decode("utf-8")

print(encrypted)
print()
print(decrypted)

òvmÔA¤-GªróSF?ß"oF0¼d!¨o85 ì÷Ñ·¦TÀ¡Kb5»J·«×£«¥]âD¾ü½²LæSþÃÝµ±LQbËÊÇ-<HÏ»$Îµ³&¯¦ÉF3i/!Yô)Ò-lÝèKèêÓ=¼1Ôâ3àÄ/ÜÊ=Û¼ÍÆáàïÔ,äñ Oæ×Õ°Å#<M~À¶S¡5ç­_ú!ÿbBÀ=J´,·d¡0ßtì;ÝÑÙä¡bOÓÁàm+O¢

Ala ma kota


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [21]:
def insert_zeros(m, l):
    size = len(m)
    binary = format(int.from_bytes(m), f'{size*8}b').replace(' ', '0')
    binary = '0' * l + binary[l:]
    num = int(binary, 2)
    return num.to_bytes(size)

In [22]:
slen = hlen
emBits = nlen - 1
emlen = nlen

def emsa_encoding(M):
    if isinstance(M, str):
        M = bytes(M, "utf-8")
    mHash = H(M).digest()
    assert emlen >= hlen + slen + 2
    salt = b''.join([random.randint(0, 255).to_bytes(1) for _ in range(slen)])
    MPrim = b'\x00' * 8 + mHash + salt
    assert len(MPrim) == 8 + hlen + slen
    mHashPrim = H(MPrim).digest()
    PS = b'\x00' * (emlen - slen - hlen - 2)
    DB = PS + b'\x01' + salt
    assert len(DB) == emlen - hlen - 1
    dbMask = mgf1(mHashPrim, emlen - hlen - 1)
    maskedDB = xor(DB, dbMask)
    maskedDB = insert_zeros(maskedDB, 8 * emlen - emBits)
    EM = maskedDB + mHashPrim + b'\xbc'
    return EM

In [23]:
def emsa_decoding(M, EM):
    if isinstance(M, str):
        M = bytes(M, "utf-8")
    mHash = H(M).digest()
    assert emlen >= hlen + slen + 2
    assert EM[-1:] == b'\xbc'
    maskedDB = EM[:emlen - hlen - 1]
    mHashPrim = EM[emlen - hlen - 1:emlen - 1]
    assert format(int.from_bytes(maskedDB), f'{len(maskedDB)*8}b').replace(' ', '0')[:8 * emlen - emBits] == '0' * (8 * emlen - emBits)
    dbMask = mgf1(mHashPrim, emlen - hlen - 1)
    DB = xor(maskedDB, dbMask)
    DB = insert_zeros(DB, 8 * emlen - emBits)
    assert DB[:emlen - hlen - slen - 2] == b'\x00' * (emlen - hlen - slen - 2)
    assert DB[emlen - hlen - slen - 2:emlen - hlen - slen - 1] == b'\x01'
    salt = DB[-slen:]
    MPrim = b'\x00' * 8 + mHash + salt
    assert len(MPrim) == 8 + hlen + slen
    mHashBis = H(MPrim).digest()
    assert mHashPrim == mHashBis


In [24]:
m = "Ala ma podpis"

em = emsa_encoding(m)
emsa_decoding(m, em)


In [25]:
def generate_SIG_emsa(M):
    em = emsa_encoding(M)
    num = int.from_bytes(em)
    SIG = pow(num, private_key[0], private_key[1])
    return (M, SIG)

def verify_SIG_emsa(SIG):
    num = pow(SIG[1], public_key[0], public_key[1])
    em = num.to_bytes(emlen)
    try:
        emsa_decoding(SIG[0], em)
    except AssertionError:
        return False
    return True

In [26]:
sig = generate_SIG_emsa(m)
verify_SIG_emsa(sig)

True

In [27]:
sig = generate_SIG_emsa(m)
sig = ("Ala nie ma podpisu", sig[1])
verify_SIG_emsa(sig)

False

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [28]:
def encrypt_key():
    r = random.randint(0, 255).to_bytes(1)
    key = H(r).digest()
    print(key)
    ciphered_key = oaep_encrypt(key, public_key)
    return ciphered_key, key

def decrypt_key(ciphered_key):
    key = oaep_decrypt(ciphered_key, private_key)
    return key

In [29]:
ciphered_key, real_key = encrypt_key()
key = decrypt_key(ciphered_key)

print(key)
assert real_key == key
print("Success!")

b'\t\xfc\x96\x08-4\xc2\xdf\xc1)]\x92\x07;^\xa1\xdc\x8e\xf8\xda\x95\xf1M\xfd\xed\x01\x1f\xfb\x96\xd3\xe5K'
b'\t\xfc\x96\x08-4\xc2\xdf\xc1)]\x92\x07;^\xa1\xdc\x8e\xf8\xda\x95\xf1M\xfd\xed\x01\x1f\xfb\x96\xd3\xe5K'
Success!
